# Quiz demo - week 2
Remember that mathematically complicated paper by Roger Shepard? We are going to implement a model of his universal law of generalization. This demo is designed more as a tutorial than an assignment, so don't fret about how much you do or don't know about coding. Hopefully, the steps below will ease you into implementing Shepard's model.

To begin with, we need to import some handy python libraries:

In [ ]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib.pylab as plt
import seaborn as sns

Next, we'll load in the data we are using for the homework. The dataset consists of 14 different colors defined by their RGB values (stored in the `color_label` array below) and their coordinate locations in CIELAB colour space. CIELAB provides a colour space that mimics how humans see colours. If you took PSY280, you might remember colour-opponent coding with one dimension representing red-green and another representing blue-yellow. CIELAB is really close to this scheme (see https://en.wikipedia.org/wiki/Lab_color_space for more info). In other words, CIELAB provides a **psychological space** of colours. 

In [ ]:
color_label = np.array([[0.4371,0,1.0000],[0.3433,0,1.0000],[0,0.3425,1.0000],[0,0.5445,1.0000],
                        [0,0.8255,0.7401],[0,0.9930,0.3840],[0,1.0000,0],[0.5963,1.0000,0],
                        [1.0000,0.7727,0],[1.0000,0.4384,0],[1.0000,0.0403,0],[1.0000,0,0],
                        [0.8170,0,0],[0.4306,0,0]])
color_pos = np.array([[-0.1530,-0.4352],[-0.1778,-0.3946],[-0.4343,-0.2730],[-0.4657,-0.2426],
                      [-0.4768,0.0639],[-0.3993,0.3136],[-0.2638,0.4160],[-0.1493,0.4715],
                      [0.2711,0.3550],[0.4004,0.1876],[0.4948,0.0357],[0.4856,-0.1096],
                      [0.4411,-0.1669],[0.4270,-0.2215]])
color_text = [str(x) for x in range(14)]

To see what this psychological space of colours looks like, run the plotting commands below:

In [ ]:
plt.scatter(color_pos[:,0],color_pos[:,1],c=color_label,s=250)
for i,txt in enumerate(color_text):
    plt.text(color_pos[i,0]+0.04,color_pos[i,1]-0.0,txt)
plt.axis('equal');
plt.xlabel('colour dimension 1')
plt.ylabel('colour dimension 2');

As you can see, CIELAB space places similar colours close to each other. We can use this psychological space to determine precisely how similar each colour is to every other color. The goal is to use Shepard's Law of Generalization to make predictions about these similarities. 

Let's start with determining the distances separating the colours.

## Euclidean distance
Euclidean distance between any two points, a and b, in 2-D space is determined by the following equation:

![](https://dl.dropboxusercontent.com/s/9dt07woa1m5clc8/euclidean_distance.png)

The function below implements this equation in python code. I have purposefully separated out the different steps of the equation so you can easily see how the math is turned into code:

In [ ]:
def euclidean_distance_2D(a,b):
    xdiff = a[0]-b[0]  # subtract the x positions for the two colours...
    xdiffsq = xdiff**2 # ...and square the difference, ** means raise to a power
    ydiff = a[1]-b[1]  # do the same thing for y positions
    ydiffsq = ydiff**2
    diffsqsum = xdiffsq+ydiffsq # add the squared x and y differences
    eucldist = diffsqsum**0.5   # and take the square root
    return(eucldist)

Let's try out the function! The distance between the first two colours (color 0 = [-0.153, -0.4352], colour 1 = [-0.1778, -0.3946], these are the two purple-ish colours at the bottom of the space) should be about 0.0476. What does the function give us?

In [ ]:
euclidean_distance_2D(color_pos[0],color_pos[1])

Great! It works! The first two colours are separated by a small distance. Let's try two colours that are on opposite sides of the colour space (i.e., purple  vs. light green):

In [ ]:
euclidean_distance_2D(color_pos[0],color_pos[7])

As expected, the Euclidean distance for these colours is much higher.

Now that we know the `euclidean_distance_2D` function is working correctly, we can calculate the difference between all pairs of colours in our space:

In [ ]:
color_dists = np.zeros((14,14))
for c1 in range(0,14):
    for c2 in range(0,14):
        color_dists[c1][c2] = euclidean_distance_2D(color_pos[c1],color_pos[c2])

One way to view all of the pairwise distances is to use a heat map plot:

In [ ]:
plt.imshow(color_dists,interpolation='nearest')
plt.title('Distance between colour pairs')
plt.xlabel('colours')
plt.ylabel('colours')
plt.colorbar(label='Euclidean distance');

Each cell in the heat map plot corresponds to a distance between a pair of colours. The "hotter" the cell (for this colour scheme is depicted as more yellow), the greater the distance; the "cooler" a cell (i.e., more blue), the shorter the distance.

At this point, we have formalized two key aspects of the model:
1. Colour is represented in a 2-dimensional space in which the dimensions correspond with the type of colours we can perceive.
2. The distance between any two colours is defined as the Euclidean distance between their positions in this colour space.

## Exponential law of generalization
We are almost there! To calculate similarity, as defined by Shepard, we need to transform distance according to the exponential law of generalization. Specifically, the similarity between a and b, given their distance, d(a,b), is defined as:

![](https://dl.dropboxusercontent.com/s/bba6x5bkmk4svsr/exponential_law.png)

Let's implement this in code. We will create a function that takes distances (`d`) as input and outputs similarity:

In [ ]:
def similarity(d):
    sims = np.exp(-d) # np.exp() is the natural exponential function  
    return(sims)

Let's test it out by comparing similarity between the two purple-ish colours (0 and 1) and the purple and light green pair (0 and 7) we looked at above:

In [ ]:
d1 = euclidean_distance_2D(color_pos[0],color_pos[1])
s1 = similarity(d1)
d2 = euclidean_distance_2D(color_pos[0],color_pos[7])
s2 = similarity(d2)
print('two purples:',s1)
print('purple vs. light green:',s2)

We should see that the two purples have a higher similarity than purple vs. light green.

With the same similarity function, we can convert all of the colour pair distances into similarities:

In [ ]:
color_sims = similarity(color_dists)
plt.imshow(color_sims,interpolation='nearest')
plt.title('Similarity between colour pairs')
plt.xlabel('colours')
plt.ylabel('colours')
plt.colorbar(label='Similarity');

This time, the cells shaded with yellow correspond with colour pairs that are very similar, cells shaded more blue correspond with different colours.

Finally, we can plot model predicted similarity as a function of distance in psychological space, similar to the Figure 1 from Shepard's paper:

In [ ]:
plt.plot(color_dists.ravel(),color_sims.ravel(),'bo') 
plt.xlabel('distance in psychological space')
plt.ylabel('model predicted similarity (generalization)')
plt.title('Shepard 1987 Similarity Predictions');

## Comparing model predictions to human colour similarity data
As a final step, we can assess the predictions of the model by comparing it to human similarity ratings for these 14 colours. Below, you'll find similarity data from a study published long ago (Ekman, 1954, Dimensions of Color Vision, Journal of Psychology). Each value corresponds to the probability that human participants labeled two colours as the same. 

In [ ]:
rating_sims = np.array([[1.00,0.74,0.36,0.26,0.16,0.06,0.06,0.05,0.04,0.07,0.07,0.11,0.15,0.12,],
                        [0.74,1.00,0.30,0.33,0.15,0.09,0.04,0.05,0.04,0.05,0.06,0.07,0.08,0.11,],
                        [0.36,0.30,1.00,0.68,0.22,0.13,0.09,0.06,0.03,0.03,0.03,0.03,0.04,0.05,],
                        [0.26,0.33,0.68,1.00,0.33,0.18,0.11,0.08,0.04,0.03,0.04,0.02,0.04,0.06,],
                        [0.16,0.15,0.22,0.33,1.00,0.49,0.24,0.22,0.07,0.04,0.03,0.03,0.04,0.04,],
                        [0.06,0.09,0.13,0.18,0.49,1.00,0.70,0.33,0.10,0.05,0.03,0.03,0.03,0.03,],
                        [0.06,0.04,0.09,0.11,0.24,0.70,1.00,0.79,0.15,0.07,0.05,0.04,0.04,0.03,],
                        [0.05,0.05,0.06,0.08,0.22,0.33,0.79,1.00,0.28,0.11,0.06,0.05,0.04,0.05,],
                        [0.04,0.04,0.03,0.04,0.07,0.10,0.15,0.28,1.00,0.55,0.24,0.15,0.15,0.11,],
                        [0.07,0.05,0.03,0.03,0.04,0.05,0.07,0.11,0.55,1.00,0.70,0.41,0.49,0.22,],
                        [0.07,0.06,0.03,0.04,0.03,0.03,0.05,0.06,0.24,0.70,1.00,0.71,0.50,0.33,],
                        [0.11,0.07,0.03,0.02,0.03,0.03,0.04,0.05,0.15,0.41,0.71,1.00,0.77,0.69,],
                        [0.15,0.08,0.04,0.04,0.04,0.03,0.04,0.04,0.15,0.49,0.50,0.77,1.00,0.95,],
                        [0.12,0.11,0.05,0.06,0.04,0.03,0.03,0.05,0.11,0.22,0.33,0.69,0.95,1.00]])

If Shepard's model accurately predicts how people judge similarity, the model predictions we calculated should match the human data above. We can assess this correspondence by plotting the model predictions along with the human similarity data.

In [ ]:
plt.plot(color_dists.ravel(),color_sims.ravel(),'bo')
plt.plot(color_dists.ravel(),rating_sims.ravel(),'rx')
plt.xlabel('distance in psychological space')
plt.ylabel('similarity')
plt.title('Model Predictions vs. Human Ratings')
plt.legend(['model','humans']);

Uh-oh, that doesn't look like a very good fit. If it was a good fit, the blue dots from the model predictions would line up well with the red xs from the human data. The model does seems to capture the overall shape (i.e., decreasing exponential function), but the scaling is off. Well, Shepard actually included a scaling parameter in the exponential function relating distance to similarity, but that was sort of glossed over in the paper. Here's the equation with this scaling parameter, `c`:

![](https://dl.dropboxusercontent.com/s/jhferwwg9jshjew/exponential_law2.png)

So, let's redefine the similarity function to include `c`:

In [ ]:
def similarity(d,c=1):
    sims = np.exp(-c*d)
    return(sims)

Now, here's where you get to interact with the model! The `c` variable below is set to the default value of 1, which we know from the simulations above doesn't fit the human data well. Try out different values of `c` in the code below and see if you can find a better fit to the human data. To do so, click on the cell, change the value of `c` (e.g., `c = 1.2`), and then run the cell. It will plot both the model and human data. Keeping on trying until you find a value for `c` that gives model predictions close to human data.

In [ ]:
# ADD CODE HERE
c = 1 # change the scaling parameter to see how it affects the model similarity predictions
color_sims = similarity(color_dists,c)

# replot the model predictions and human data
plt.plot(color_dists.ravel(),color_sims.ravel(),'bo')
plt.plot(color_dists.ravel(),rating_sims.ravel(),'rx')
plt.xlabel('distance in psychological space')
plt.ylabel('similarity')
plt.title('Model Predictions vs. Human Ratings')
plt.legend(['model','humans']);

Once you have found a scaling parameter value that gives a good fit, you're finished! Make sure to put your best fitting c value in the Quercus quiz!